In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon")
bsObj = BeautifulSoup(html, "html.parser")
for link in bsObj.findAll("a"):
    if 'href' in link.attrs:
        print(link.attrs['href'])

In [ ]:
import re

In [ ]:
for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a",
                        href=re.compile("^(/wiki/)((?!:).)*$")):
    if 'href' in link.attrs:
        print(link.attrs['href'])

In [ ]:
import datetime
import random
random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    html = urlopen("http://en.wikipedia.org"+articleUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a",
                        href=re.compile("^(/wiki/)((?!:).)*$"))
links = getLinks("/wiki/Kevin_Bacon")
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)
    

In [9]:
pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "html.parser")
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # 새 페이지를 발견
                newPage = link.attrs['href']
                print("----------------\n"+newPage)
                pages.add(newPage)
                getLinks(newPage)
            
getLinks("")

----------------
/wiki/Main_Page
----------------
/wiki/Wikipedia:Contents
----------------
/wiki/Portal:Current_events
----------------
/wiki/Special:Random
----------------
/wiki/Wikipedia:About
----------------
/wiki/Help:Contents
----------------
/wiki/Help:Introduction
----------------
/wiki/Wikipedia:Community_portal
----------------
/wiki/Special:RecentChanges
----------------
/wiki/Wikipedia:File_upload_wizard
----------------
/wiki/Special:Search
----------------
/wiki/Special:MyContributions
----------------
/wiki/Special:MyTalk
----------------
/wiki/Special:WhatLinksHere/User_talk:103.116.72.25


HTTPError: HTTP Error 404: Not Found

In [11]:
pages = set()
random.seed(datetime.datetime.now())

def getIntLinks(bsObj, includeUrl):
    internalLinks = []
    for link in bsObj.findAll("a", href=re.compile("^(/|.*"+includeUrl+")")): # 正则表达式: 以/开头或者包含includeUrl
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith("/")):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    for link in bsObj.findAll("a",
                        href=re.compile("^(http|www)((?!"+excludeUrl+").)*$")): # 正则表达式: 以http或www开头, 但不包含excludeUrl
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def splitAddress(address):
    addressParts = address.replace("http://", "").split("/") 
    return addressParts

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bsObj = BeautifulSoup(html, "html.parser")
    externalLinks = getExternalLinks(bsObj, splitAddress(startingPage)[0]) # splitAddress(startingPage)[0] = domain
    if len(externalLinks) == 0:
        domain = splitAddress(startingPage)[0]
        internalLinks = getIntLinks(bsObj, domain)
        return getRandomExternalLink(internalLinks[random.randint(0,
                                                len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]
    
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print("Random external link is: "+externalLink)
    followExternalOnly(externalLink)

followExternalOnly("http://oreilly.com")


/var/folders/6l/0rnqwtgd2lqfvr4vcry4xjrc0000gn/T/ipykernel_31421/2675707815.py:2: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


Random external link is: https://play.google.com/store/apps/details?id=com.safariflow.queue
Random external link is: https://support.google.com/googleplay/answer/7007852
Random external link is: https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=http://support.google.com/googleplay/answer/7007852&ec=GAZAdQ
Random external link is: https://support.google.com/accounts?p=signin_privatebrowsing&hl=en
Random external link is: https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=http://support.google.com/accounts/answer/2917834%3Fvisit_id%3D638248605906464470-1041951599%26p%3Dsignin_privatebrowsing%26hl%3Den%26rd%3D1&ec=GAZAdQ
Random external link is: https://support.google.com/accounts?p=signin_privatebrowsing&hl=en
Random external link is: https://support.microsoft.com/en-us/help/4026200/windows-browse-inprivate-in-microsoft-edge
Random external link is: https://support.xbox.com/


ValueError: empty range for randrange() (0, 0, 0)

In [12]:
allExtLinks = set()
allIntLinks = set()
def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = splitAddress(siteUrl)[0]
    bsObj = BeautifulSoup(html, "html.parser")
    internalLinks = getIntLinks(bsObj, domain)
    externalLinks = getExternalLinks(bsObj, domain)
    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
    for link in internalLinks:
        if link not in allIntLinks:
            print("About to get link: "+link)
            allIntLinks.add(link)
            getAllExternalLinks(link)

getAllExternalLinks("http://oreilly.com")

https://twitter.com/oreillymedia
https://www.linkedin.com/company/oreilly-media
https://www.youtube.com/user/OreillyMedia
https://oreilly.hk/
https://oreillylearning.in/
https://oreilly.id/
https://www.oreilly.co.jp/index.shtml
https://itunes.apple.com/us/app/safari-to-go/id881697395
https://play.google.com/store/apps/details?id=com.safariflow.queue
https://channelstore.roku.com/details/c8a2d0096693eb9455f6ac165003ee06/oreilly
https://www.amazon.com/OReilly-Media-Inc/dp/B087YYHL5C/ref=sr_1_2?dchild=1&keywords=oreilly&qid=1604964116&s=mobile-apps&sr=1-2
About to get link: https://www.oreilly.com
https://www.oreilly.com
https://www.oreilly.com/member/login/
https://www.oreilly.com/online-learning/try-now.html
https://www.oreilly.com/online-learning/teams.html
https://www.oreilly.com/online-learning/government.html
https://www.oreilly.com/online-learning/academic.html
https://www.oreilly.com/online-learning/individuals.html
https://www.oreilly.com/online-learning/features.html
https://www

URLError: <urlopen error no host given>

In [14]:
from scrapy import Item, Field,Selector,Spider
class Article(Item):
    title = Field()
    text = Field()

class ArticleSpider(Spider):
    name = "article"
    allowed_domains = ["en.wikipedia.org"]
    start_urls = ["http://en.wikipedia.org/wiki/Main_Page",
                  "http://en.wikipedia.org/wiki/Python_%28programming_language%29"]
    def parse(self, response):
        item = Article()
        title = response.xpath('//h1/text()')[0].extract()
        print("Title is: "+title)
        item['title'] = title
        return item
    

